In [ ]:
import random
from random import seed
from random import randint
random.seed(109)

In [ ]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        if(self.current_player == 1):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[pit-1] > 0): 
                return True
            else:
                return False
        elif(self.current_player == 2):
            if (pit > 0 and pit < (self.pits_per_player + 1)) and (self.board[self.pits_per_player + pit] > 0): 
                return True
            else:
                return False
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here
        value = randint(1, self.pits_per_player)
        while(self.board[value + self.pits_per_player] == 0):
            value = randint(1, self.pits_per_player)
        return value
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        # write your code here
        capturePlayer1 = False
        capturePlayer2 = False
        if (self.valid_move(pit)): 
            print("Player", self.current_player, "selected pit", pit)
            if(self.current_player == 1):
                front = pit
                end = pit + self.board[pit - 1]
                self.board[pit - 1] = 0
                for i in range(front, end):
                    if (i == (end - 1)) and (i > 0 and i < (self.pits_per_player)) and (self.board[i] == 0):
                        val = self.board[self.pits_per_player * 2 - i]
                        self.board[self.pits_per_player * 2 - i] = 0
                        self.board[self.pits_per_player] = self.board[self.pits_per_player] + val
                        capturePlayer1 = True
                    else:
                        if capturePlayer1 and i == (end - 1):
                            continue
                        else:
                            self.board[i] = self.board[i] + 1
                if capturePlayer1:
                    self.board[self.pits_per_player] = self.board[self.pits_per_player] + 1
                    capturePlayer1 = False
                add = ("Player 1 selected pit", pit)
                self.moves.append(add)
                self.current_player = 2
            elif(self.current_player == 2):
                # print("front", pit + self.pits_per_player + 1)
                # print("end", pit + self.pits_per_player + 1 + self.board[pit + 6])
                numMoves = self.board[pit + self.pits_per_player]
                front = pit + self.pits_per_player + 1
                end = pit + self.pits_per_player + 1 + self.board[pit + self.pits_per_player]
                self.board[pit + self.pits_per_player] = 0
                for i in range(front, end):
                    if (i == (end - 1)) and (i > self.pits_per_player and i < (self.pits_per_player * 2 + 1)) and (self.board[i] == 0):
                        val = self.board[self.pits_per_player * 2 - i]
                        self.board[self.pits_per_player * 2 - i] = 0
                        self.board[self.pits_per_player * 2 + 1] = self.board[self.pits_per_player * 2 + 1] + val
                        capturePlayer2 = True
                    if i > (self.pits_per_player * 2 + 1):
                        for j in range(0, numMoves - ((self.pits_per_player * 2 + 1) - (self.pits_per_player + pit))):
                            self.board[j] = self.board[j] + 1
                        add = ("Player 2 selected pit", pit)
                        self.moves.append(add)
                        self.current_player = 1
                        return
                    else:
                        if capturePlayer2 and i == (end - 1):
                            continue
                        else:
                            self.board[i] = self.board[i] + 1
                if capturePlayer2:
                    self.board[self.pits_per_player * 2 + 1] = self.board[self.pits_per_player * 2 + 1] + 1
                    capturePlayer2 = False
                add = ("Player 2 selected pit", pit)
                self.moves.append(add)
                self.current_player = 1
        else:
            print("Player", self.current_player ,"selected pit", pit)
            print("INVALID MOVE")
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        # write your code here
        for i in range(len(self.board)):
            if(self.board[i] == 0 and i != self.pits_per_player and i != (self.pits_per_player * 2 + 1)):
                continue
            else:
                return False
        return True